In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data=pd.read_csv("spam_data_set.csv", encoding="latin-1")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
#data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
#data.drop(['class'], axis=1, inplace=True)


In [ ]:
data.head()

In [ ]:
data.rename(columns={'Category': 'class','Message':'message'}, inplace=True)

In [ ]:
data['class']=data['class'].map({'ham':0, 'spam':1})

In [ ]:
data.head()

In [ ]:
data.dropna(inplace=True)  

In [ ]:
# Remove duplicates
data.drop_duplicates(inplace=True)

In [ ]:
data['message'] = data['message'].str.lower()
data.head()

In [91]:
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



# Handle missing values
data.dropna(inplace=True)  # Drop rows with missing values

# Remove duplicates
data.drop_duplicates(inplace=True)

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Download NLTK resources (run this once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to clean and lemmatize text
def preprocess_text(text):
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text.lower())  # Lowercase during tokenization
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

# Apply the preprocessing function to the message column
data['message'] = data['message'].apply(preprocess_text)

# Display the processed dataframe
print(data.head())

   class                                            message
0      0  go jurong point crazy available bugis n great ...
1      0                            ok lar joking wif u oni
2      1  free entry 2 wkly comp win fa cup final tkts 2...
3      0                u dun say early hor u c already say
4      0           nah dont think go usf life around though


In [92]:
from sklearn.model_selection import train_test_split

In [93]:
X=data['message']
y=data['class']

In [94]:
X.shape

(5156,)

In [95]:
y.shape

(5156,)

In [96]:
data.isnull().sum()

class      0
message    0
dtype: int64

In [97]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer()

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv= TfidfVectorizer(
            max_features=3000,  # Slightly reduced feature set
            ngram_range=(1, 2),  # Unigrams and bigrams
            stop_words='english',
            lowercase=True
        )


In [104]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)

In [105]:
#x_train, x_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42,stratify=y)



In [106]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

# Train Naive Bayes model
nb_model = MultinomialNB(alpha=0.1)
nb_model.fit(x_train, y_train)

# Train Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train, y_train)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Predictions and accuracy evaluation for Naive Bayes
y_pred_nb = nb_model.predict(x_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Multinomial Naive Bayes Model Accuracy: {accuracy_nb:.2f}")

# Predictions and accuracy evaluation for Logistic Regression
y_pred_lr = lr_model.predict(x_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Model Accuracy: {accuracy_lr:.2f}")

# Predictions and accuracy evaluation for Random Forest
y_pred_rf = rf_model.predict(x_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Model Accuracy: {accuracy_rf:.2f}")

# Save the models and vectorizer
# with open('spam_nb.pkl', 'wb') as nb_file:
#     pickle.dump(nb_model, nb_file)
pickle.dump(nb_model, open('spam_nb.pkl','wb'))

# with open('spam_lr.pkl', 'wb') as lr_file:
#     pickle.dump(lr_model, lr_file)
pickle.dump(lr_model, open('spam_lr.pkl','wb'))

# with open('spam_rf.pkl', 'wb') as rf_file:
#     pickle.dump(rf_model, rf_file)
pickle.dump(rf_model, open('spam_rf.pkl','wb'))


# with open('vectorizer.pkl', 'wb') as vectorizer_file:
#     pickle.dump(cv, vectorizer_file)  
pickle.dump(cv, open('vectorizer.pkl','wb'))  

Multinomial Naive Bayes Model Accuracy: 0.98
Logistic Regression Model Accuracy: 0.96
Random Forest Model Accuracy: 0.97
